In [1]:
import os

import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [15]:
# Set parameter values
PLOTS_DIR = "../results/plots/"

X_TITLE = "Margin"
Y_TITLE = "Annotations / Data Point"

In [3]:
# Load data
results = pd.read_csv("../results/results_full.csv")
results["gap"] = results.apply(lambda row: f"{round(row['gap_rmse'], 3)}", axis=1)
results_df = results[["k", "m", "q", "p", "eps", "rank_success", "gap"]]

**Generating Plots for All Parameter Combinations**

In [ ]:
for k in set(results_df["k"].to_list()):
    for q in set(results_df["q"].to_list()):
        for p in set(results_df["p"].to_list()):
            df = results_df[
                (results_df["k"] == k) & (results_df["q"] == q) & (results_df["p"] == p)
            ]

            df["m"] = df["m"].astype(str)
            rank_success_pivot = df.pivot(
                index="m", columns="eps", values="rank_success"
            )
            gap_pivot = df.pivot(index="m", columns="eps", values="gap")
            m_order = sorted(df["m"].astype(float).unique())
            m_order = [str(int(m)) if m.is_integer() else str(m) for m in m_order]
            rank_success_pivot = rank_success_pivot.loc[m_order]
            gap_pivot = gap_pivot.loc[m_order]

            heatmap_rank_success = go.Figure(
                data=go.Heatmap(
                    z=rank_success_pivot.loc[m_order].values,
                    x=rank_success_pivot.columns,
                    y=rank_success_pivot.index,
                    zmin=0.5,
                    zmax=1,
                    text=rank_success_pivot.values.round(3).astype(str),
                    texttemplate="%{text}",
                    colorbar_title="Rank Success",
                    textfont={"size": 12},
                    showscale=False,
                )
            )
            heatmap_rank_success.update_layout(
                title={
                    "text": f"Raking Success (Budget: {k}, Annotator Accuracy: {q}, Base Accuracy: {p})",
                    "x": 0.5,
                    "xanchor": "center",
                    "font": {"size": 13},
                },
                xaxis_title=X_TITLE,
                yaxis_title=Y_TITLE,
                margin=dict(t=50, b=0, l=0, r=10),
                height=300,
                width=500,
            )

            # Heatmap for gap
            heatmap_gap = go.Figure(
                data=go.Heatmap(
                    z=[
                        [float(val.split(" ± ")[0]) for val in row]
                        for row in gap_pivot.loc[m_order].values
                    ],
                    x=gap_pivot.columns,
                    y=gap_pivot.index,
                    text=gap_pivot.values,
                    reversescale=True,
                    texttemplate="%{text}",
                    colorbar_title="Gap",
                    textfont={"size": 12},
                    showscale=False,
                )
            )
            heatmap_gap.update_layout(
                title={
                    "text": f"Gap (Budget: {k}, Annotator Accuracy: {q}, Base Accuracy: {p})",
                    "x": 0.5,
                    "xanchor": "center",
                    "font": {"size": 13},
                },
                xaxis_title=X_TITLE,
                yaxis_title=Y_TITLE,
                margin=dict(t=50, b=0, l=0, r=10),
                height=300,
                width=500,
            )

            pio.write_image(
                heatmap_rank_success,
                os.path.join(PLOTS_DIR, f"rank-k={k}-q={q}-p={p}.pdf"),
                format="pdf",
            )
            pio.write_image(
                heatmap_gap,
                os.path.join(PLOTS_DIR, f"gap-k={k}-q={q}-p={p}.pdf"),
                format="pdf",
            )

In [ ]:
unique_k_values = set(results_df["k"].to_list())

# Manually specify values to display
unique_p_values = [0.6, 0.75, 0.9]
unique_q_values = [0.6, 0.75, 0.9]

os.makedirs(PLOTS_DIR, exist_ok=True)

for k in unique_k_values:
    fig = make_subplots(
        rows=len(unique_q_values),
        cols=len(unique_p_values),
        vertical_spacing=0.05,
        horizontal_spacing=0.02,
    )

    for i, q in enumerate(unique_q_values, start=1):
        for j, p in enumerate(unique_p_values, start=1):
            df = results_df[
                (results_df["k"] == k) & (results_df["q"] == q) & (results_df["p"] == p)
            ]

            if df.empty:
                continue

            df["m"] = df["m"].astype(str)

            rank_success_pivot = df.pivot(
                index="m", columns="eps", values="rank_success"
            )

            m_order = sorted(df["m"].astype(float).unique())
            m_order = [str(int(m)) if m.is_integer() else str(m) for m in m_order]
            rank_success_pivot = rank_success_pivot.loc[m_order]

            z_values = rank_success_pivot.loc[m_order].values
            text_values = rank_success_pivot.values

            heatmap = go.Heatmap(
                z=z_values,
                x=rank_success_pivot.columns,
                y=m_order,
                showscale=False,
                text=text_values,
                texttemplate="%{text}",
                textfont={"size": 13},
            )
            fig.add_trace(heatmap, row=i, col=j)

    fig.update_layout(
        height=200 * len(unique_q_values),
        width=500 * len(unique_p_values),
        margin=dict(t=10, b=10, l=0, r=0),
    )

    for i in range(1, len(unique_q_values) + 1):
        fig.update_yaxes(
            title_text=f"Annotator Acc.: {unique_q_values[i-1]}",
            row=i,
            col=1,
            ticks="outside",
        )
    for j in range(1, len(unique_p_values) + 1):
        fig.update_xaxes(
            title_text=f"Better Classifier Acc.: {unique_p_values[j-1]}",
            row=len(unique_q_values),
            col=j,
            ticks="outside",
        )

    output_file = os.path.join(PLOTS_DIR, f"rank-heatmap_of_heatmaps_k={k}.pdf")
    fig.write_image(output_file, format="pdf")

    # Plots for ranking
    fig = make_subplots(
        rows=len(unique_q_values),
        cols=len(unique_p_values),
        vertical_spacing=0.05,
        horizontal_spacing=0.02,
    )

    for i, q in enumerate(unique_q_values, start=1):
        for j, p in enumerate(unique_p_values, start=1):
            df = results_df[
                (results_df["k"] == k) & (results_df["q"] == q) & (results_df["p"] == p)
            ]

            if df.empty:
                continue

            df["m"] = df["m"].astype(str)

            gap_pivot = df.pivot(index="m", columns="eps", values="gap")

            m_order = sorted(df["m"].astype(float).unique())
            m_order = [str(int(m)) if m.is_integer() else str(m) for m in m_order]
            gap_pivot = gap_pivot.loc[m_order]

            z_values = [
                [float(val.split(" ± ")[0]) for val in row] for row in gap_pivot.values
            ]
            text_values = gap_pivot.values

            heatmap = go.Heatmap(
                z=z_values,
                x=gap_pivot.columns,
                y=m_order,
                showscale=False,
                reversescale=True,
                text=text_values,
                texttemplate="%{text}",
                textfont={"size": 13},
            )
            fig.add_trace(heatmap, row=i, col=j)

    fig.update_layout(
        height=200 * len(unique_q_values),
        width=500 * len(unique_p_values),
        margin=dict(t=10, b=10, l=0, r=0),
    )

    for i in range(1, len(unique_q_values) + 1):
        fig.update_yaxes(
            title_text=f"Annotator Acc.: {unique_q_values[i-1]}",
            row=i,
            col=1,
            ticks="outside",
        )
    for j in range(1, len(unique_p_values) + 1):
        fig.update_xaxes(
            title_text=f"Better Classifier Acc.: {unique_p_values[j-1]}",
            row=len(unique_q_values),
            col=j,
            ticks="outside",
        )

    output_file = os.path.join(PLOTS_DIR, f"heatmap_of_heatmaps_k={k}.pdf")
    fig.write_image(output_file, format="pdf")